# Segmenting Cardiomyoctye Model Parameters

### *Personal Note*

*The data and analysis included here was part of a project where I evaluated predictions of cardiomyocyte action potential models. The objective here is to separate different hypotheses from a population of models. Additionally, I review some different clustering stratagies. The model features have biophysical and physiological meaning, but I will put a principal component step in as well. The data set here was the resulting output from training multiple populations of models using evolutionary optimization algorithms. I hope you find it interesting.*

## Introduction

The cardiac action potential (AP) is the electrical waveform that is conducted through individual cells during a cardiac cycle. Cardiac electrophysiology is a complex system consisting of electrically active proteins, organelles, and tonal signalling molecules that work in conjuction to produce the AP. Each singular cell expresses different quantities of each active component, which will influence the risk of an arrhythmia occurance. It is quite challenging to determine the component quanties in individual cells, so it is useful to train AP models on data sets collected from specific cells to estimate these quantities. I used an evolutionary algorithm (EA) to optimize 14 parameters and pooled the best performing models into a single dataset. In this analysis I am going to use clustering algorithms to segment the proposed solutions from the EA. I expect that independent EA optimizations will produce independent solutions, but if multiple EA runs converge on the same solution then it might suggest that a global minimum was discovered.